<a href="https://colab.research.google.com/github/Yooru6/TensorFlow---GenerateText/blob/master/Word_based_Generate_Anime_Bidirectional_ManyToOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GENERATE ANiME - BIDIRECTIONAL MANY TO ONE MODEL

In [1]:
import os

%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.layers import LSTM, Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.callbacks import *

#!pip install -q -U tensorflow>=1.7
print(tf.__version__)
import pandas as pd
import numpy as np
!pip install -q pyyaml h5py  # Required to save models in HDF5 format
from __future__ import absolute_import, division, print_function, unicode_literals
from nltk.tokenize import RegexpTokenizer
import pprint
from collections import Counter
import re
#from sklearn.utils import shuffle

#!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

TensorFlow 2.x selected.
Tensorflow version 2.1.0
2.1.0


Using TensorFlow backend.


In [0]:
#Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Download Dataset from google drive

In [3]:
link='https://drive.google.com/open?id=1-X0LsGR2RgXBJ4YGbnLgNHKd0z9i8Fs0'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1-X0LsGR2RgXBJ4YGbnLgNHKd0z9i8Fs0


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ultimatePlot.txt')
text=(open("ultimatePlot.txt").read())
text=text.lower()
#print(text[:250190])

## Replace Words/chars

In [0]:
#Replace chars/words
#textedited=text[:450000]
#textedited=text[:290000]
textedited = text.replace("ultimate:\n","")#normaalisti text.replace tässä kohtaan
textedited = textedited.replace("n/a","")
textedited = textedited.replace("N/A","")
textedited = textedited.replace("<em class=de-emphasized>(from manga)</em>","")
textedited = textedited.replace("<em>","")
textedited = textedited.replace("</em>","")
textedited = textedited.replace("<i>","")
textedited = textedited.replace("</i>","")
textedited = textedited.replace("<cite>","")
textedited = textedited.replace("</cite>","")

## Split and index words manually

In [7]:
from nltk.tokenize import WordPunctTokenizer

#Tokenize text
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
tk = WordPunctTokenizer() 
text2=tk.tokenize(textedited)

#Dictionary words etc.
countsT=Counter()
countsT.update(text2)
words=sorted(countsT,key=countsT.get,reverse=True)
nb_words=len(text2)
word2index={word: i  for i, word in enumerate(words)} # word/num
index2word={i: word for i, word in enumerate(words)} # num/word

print(len(words)) #sanat jotka esiintyy tekstissä
print(nb_words)#Sanojen määrä yhteensä koko tekstissä
print(len(word2index))#Word : index
#print(index2word)#index num : word



29457
517569
29457


## Make X and Y value lists

In [8]:
#X and Y vector data. (X=1234,Y=5; X=2345,Y=6; X=3456,Y=7. etc..)
sequence_length=4
step=1
def makeDataSet():
  global x_values
  global sequence_length
  
  x_list=[]
  y_list=[]

  #POPULATE LISTS
  for i in range(0,nb_words-sequence_length,step):
    #print(i)
    x=text2[i:(i+sequence_length)]
    y=text2[i+sequence_length]
    x_list.append(x)
    y_list.append(y)
  return x_list,y_list

x_list,y_list = makeDataSet()
    
print(len(x_list),' ',len(y_list))


517565   517565


In [9]:
#CHECK
print(x_list[1])
print(y_list[1])

#TRAINING DATA
x_list=x_list[:150000]
y_list=y_list[:150000]

#TEST DATA
x_test=x_list[300001:320000]
y_test=y_list[300001:320000]

['okabe', 'is', 'a', 'self']
mad
150000
150000


## Make boolean vectors

In [10]:
total_words=len(set(words))
x_train = np.zeros((len(x_list),sequence_length,total_words),dtype=np.bool)

print(total_words)

29457


In [0]:
y_train = np.zeros((len(x_list),total_words),dtype=np.bool)

## Make training set by populatining values to boolean lists
  

In [12]:
print(x_train.shape, " ",y_train.shape)
for i, input_word in enumerate(x_list):
  for j, word in enumerate(input_word):
   # print(j)
    x_train[i,j,word2index[word]]=1
  y_train[i,word2index[y_list[i]]]=1
  #print(i)
#print(y_train[1])

(150000, 4, 29457)   (150000, 29457)


## MODEL

In [0]:
HIDDEN_SIZE=500
HIDDEN_SIZE2=500
HIDDEN_SIZE3=512
BATCH_SIZE=256
NUM_ITERATIONS=100
NUM_EPOCHS_PER_ITERATION= 1
NUM_PREDS_PER_EPOCHS=100
EPOCHS_NUM=40

def create_model():
  model=Sequential()
  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(HIDDEN_SIZE,return_sequences=True),input_shape=(sequence_length,total_words)))
  model.add(tf.keras.layers.Dropout(0.4))
  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM((HIDDEN_SIZE2))))
  model.add(tf.keras.layers.Dropout(0.4))
  #model.add(tf.keras.layers.Flatten(data_format=None))
  model.add(tf.keras.layers.Dense(total_words,activation="softmax"))
  model.compile(
  optimizer='adam',
  loss = 'categorical_crossentropy',
  metrics=['accuracy']
  )

  model.summary()
  return model

In [14]:

model = create_model()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 4, 1000)           119832000 
_________________________________________________________________
dropout (Dropout)            (None, 4, 1000)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000)              6004000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense (Dense)                (None, 29457)             29486457  
Total params: 155,322,457
Trainable params: 155,322,457
Non-trainable params: 0
_________________________________________________________________


### Model:
  textedited[:10000] rajattu data setti<br>
  Model1:
  LSTM(512)<br>
  LSTM(256, activation: relu)<br>
  dense(output,actiuvation=softmax)<br>
  batch=128<br>
  total_epochs=50<br>
  Generating from seed: ['child', 'with', 'a', 'younger', 'lover.', 'the', 'rest', 'of', 'his', 'family']
he ask to his months disobeying future his his replace and as on bounty, tomoe. tomoya. with and search for with many bandits in to love to to village jinta born the a he way to their they sent involves 

In [15]:
drive.mount('/content/gdrive')

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# download to google drive
#drive.CreateFile({'id': model_file.get('id')})


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## FIT MODEL:

In [0]:
#x_train, y_train = shuffle(x_train, y_train)

#CALLBACKS
filepath="/content/gdrive/My Drive/Colab Notebooks/checkpoints/ANiME_2x500BIdrctl_epochs:{epoch:03d}-loss:{loss:.3f}-seqL:"+str(sequence_length)+".h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=True, period=5)
callbacks_list = [checkpoint]

#LOAD MODEL
filepath_load="/content/gdrive/My Drive/Colab Notebooks/checkpoints/ANiME_2x500BIdrctl_epochs:010-loss:0.809-seqL:4.h5"
#model.load_weights(filepath_load)

# FIT MODEL AND PREDICT AFTER X EPOCHS
for iteration in range(10):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(x_train, y_train, epochs=EPOCHS_NUM,validation_split=0.05, batch_size=BATCH_SIZE,verbose=1,callbacks=callbacks_list,shuffle=True)#4811 0.0923393
    test_idx = np.random.randint(int(len(x_list)*0.1)) * (-1)
    test_words = x_list[test_idx]
    print("Generating from seed: %s" % (test_words))
    for i in range(EPOCHS_NUM*8):        
        Xtest = np.zeros((1, sequence_length, total_words))
        for i, ch in enumerate(test_words):
            Xtest[0, i, word2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]
        print(ypred,end=' ')
        if (ypred=="." or ypred == "!" or ypred == "?"):
          print('')
        test_words = test_words[1:] + [ypred]


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Iteration #: 0
Train on 142500 samples, validate on 7500 samples
Epoch 1/40
142500/142500 [==============================] - 106s 745us/sample - loss: 0.8420 - accuracy: 0.7871 - val_loss: 8.5912 - val_accuracy: 0.1527
Epoch 2/40
142500/142500 [==============================] - 95s 670us/sample - loss: 0.7570 - accuracy: 0.8103 - val_loss: 8.6897 - val_accuracy: 0.1509
Epoch 3/40
142500/142500 [==============================] - 95s 670us/sample - loss: 0.6746 - accuracy: 0.8274 - val_loss: 8.9203 - val_accuracy: 0.1505
Epoch 4/40
142500/142500 [==============================] - 95s 669us/sample - loss: 0.6175 - accuracy: 0.8411 - val_loss: 9.1777 - val_accuracy: 0.1521
Epoch 5/40
142336/142500 [============================>.] - ETA: 0s - loss: 0.5666 - accuracy: 0.8524
Epoch 00005: saving model to /content/gdrive/My Drive/Colab Notebooks/checkpoints/ANiME_2

## NOTES:
start:

#### TRY 1:<br>
batchsize 256<br>
/content/gdrive/My Drive/Colab Notebooks/checkpoints/ANiME_2x500BIdrctl_epochs:005-loss:4.781-seqL:4.h5:<br>
10 epochs: .<br>

Generating from seed: ['of', 'judoh', ',', 'claire']
leonelli has inherited leadership of the mafia group " vampire " following the death of his family . 
in the city of shinjuku , a conflict between the two . 
the sequel is set off to save yuzu from shuren ( the commander of the denizens that kidnapped ichigo ' s sisters ). it is the way to find her in order to protect wako agemaki , girl who saved his life and is one of the four

30 epochs:<br>
Generating from seed: ['are', 'you', 'going', 'to']
be living with them , into a respectable young lady . 
she agrees to make their living expenses free of charge if they succeed , but if they fail , she ' ll triple their rent . 
due to their confidence in knowing how to deal with women , they agree to make a contract with a strange boy named goku . 
the two then set off together , bulma in search of the dragonballs and goku on a quest to free all the minds trapped in aincrad . 
grimm masterpiece theatre was japanese interpretations of the old grimm ' s fairy tales . 
this series covers fairy tales that are well known to lesser known tales like " puss ' n boots ", " snow white ", " cinderella ", and " hansel and gretel ". this is the story of the medicine seller from the " bakeneko " arc of " ayakashi ," as he continues to face various dangerous spirits . 
takashi natsume sees spirits , and with the help of a ninja girl and a mysterious raven hair woman . 







## TEST MODELS:

In [39]:
 filepath_load="/content/gdrive/My Drive/Colab Notebooks/checkpoints/ANiME_2x500BIdrctl_epochs:030-loss:0.253-seqL:4.h5"
 model.load_weights(filepath_load)
  
test_idx = np.random.randint(int(len(x_list)*0.2)) * (-1)
test_words = ["are","you","going","to"]#x_list[test_idx]
print("Generating from seed: %s" % (test_words))
for i in range(EPOCHS_NUM*5):        
    Xtest = np.zeros((1, sequence_length, total_words))
    for i, ch in enumerate(test_words):
        Xtest[0, i, word2index[ch]] = 1
    pred = model.predict(Xtest, verbose=0)[0]
    ypred = index2word[np.argmax(pred)]
    print(ypred,end=' ')
    if (ypred=="." or ypred == "!" or ypred == "?"):
      print('')
    #remove first word from test list and add predicted word to end of the list
    test_words = test_words[1:] + [ypred]



Generating from seed: ['are', 'you', 'going', 'to']
be living with them , into a respectable young lady . 
she agrees to make their living expenses free of charge if they succeed , but if they fail , she ' ll triple their rent . 
due to their confidence in knowing how to deal with women , they agree to make a contract with a strange boy named goku . 
the two then set off together , bulma in search of the dragonballs and goku on a quest to free all the minds trapped in aincrad . 
grimm masterpiece theatre was japanese interpretations of the old grimm ' s fairy tales . 
this series covers fairy tales that are well known to lesser known tales like " puss ' n boots ", " snow white ", " cinderella ", and " hansel and gretel ". this is the story of the medicine seller from the " bakeneko " arc of " ayakashi ," as he continues to face various dangerous spirits . 
takashi natsume sees spirits , and with the help of a ninja girl and a mysterious raven hair woman . 
500 years before saiyuki , a 

## SAVE MODEL

In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# create on Colab directory
model.save_weights('ESTRY_2x400BIdrctl_epochs:001-loss:0.388--seqL:4.h5')    
model_file = drive.CreateFile({'title' : 'ESTRY_2x400BIdrctl_epochs:001-loss:0.388--seqL:4.h5'})
model_file.SetContentFile('ESTRY_2x400BIdrctl_epochs:001-loss:0.388--seqL:4.h5')
model_file.Upload()
# download to google drive
drive.CreateFile({'id': model_file.get('id')})